In [0]:
from pyspark.sql import SparkSession
import pandas as pd
spark = SparkSession.builder.getOrCreate()
df_spark = spark.table("labeled_step_test")
df = df_spark.toPandas()
df.head()

In [0]:
feature_cols_numeric = ["distance_cm"]
feature_cols_categorical = ["sensor_type", "device_id"]
label_col = "step_label"

In [0]:
from sklearn.model_selection import train_test_split
X = df[feature_cols_numeric + feature_cols_categorical]
y = df[label_col]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [0]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

numeric_transformer = StandardScaler()

In [0]:
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

In [0]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, feature_cols_numeric),
        ("cat", categorical_transformer, feature_cols_categorical)
    ]
)

In [0]:
from sklearn.pipeline import Pipeline
pipeline = Pipeline(steps=[
    ("preprocess", preprocessor)
])

In [0]:
pipeline.fit(X_train)

X_train_transformed = pipeline.transform(X_train)
X_test_transformed = pipeline.transform(X_test)

In [0]:
import joblib
import os

base_path = "/tmp/etl_pipeline/"

if not os.path.exists(base_path):
    os.makedirs(base_path)
    print(f"Created directory: {base_path}")

joblib.dump(pipeline, os.path.join(base_path, "stedi_feature_pipeline.pkl"))
joblib.dump(X_train_transformed, os.path.join(base_path, "X_train_transformed.pkl"))
joblib.dump(X_test_transformed, os.path.join(base_path, "X_test_transformed.pkl"))
joblib.dump(y_test, os.path.join(base_path, "y_test.pkl"))
joblib.dump(y_train, os.path.join(base_path, "y_train.pkl"))

print("All files saved successfully to /tmp/etl_pipeline/!")

##Ethics Reflection

Using a consistent, reproducible feature pipeline prevents unfairness by ensuring that every data point, regardless of its source, is treated with the exact same mathematical logic. In Machine Learning, "hidden bias" often creeps in when we process different groups of data inconsistently, but a pipeline locks our preprocessing (like scaling and encoding) into a stable standard. This technical consistency mirrors the spiritual principle of Equity, as taught in the scriptures: God is "no respecter of persons" (Acts 10:34) and operates by unchanging laws. By building reliable pipelines, we ensure our models do not favor certain device types or demographics due to sloppy or varied data handling. Just as consistent spiritual habits build a stable foundation, consistent data habits build trustworthy and fair AI systems.

In [0]:
import joblib
import joblib
import numpy as np
import pandas as pd
from scipy.sparse import issparse

base_path = "/tmp/etl_pipeline/"

pipeline = joblib.load(base_path + "stedi_feature_pipeline.pkl")
X_train_transformed = joblib.load(base_path + "X_train_transformed.pkl")
X_test_transformed = joblib.load(base_path + "X_test_transformed.pkl")
y_train = joblib.load(base_path + "y_train.pkl")
y_test = joblib.load(base_path + "y_test.pkl")

def to_float_matrix(arr: np.ndarray) -> np.ndarray:
    """
    Ensures that input arrays (possibly object-dtype, sparse, or 0-d) are converted to a 2-D float matrix.
    This is necessary because saved feature arrays may have inconsistent shapes or types after transformation,
    and ML models require numeric 2-D arrays for training and prediction.
    """
    if arr.ndim == 0:
        arr = arr.item()
        if issparse(arr):
            arr = arr.toarray()
        arr = np.array(arr, dtype=float)
    elif arr.dtype == object:
        arr = np.array([
            x.toarray() if issparse(x) else np.array(x, dtype=float)
            for x in arr
        ])
        arr = np.vstack(arr)
    elif issparse(arr):
        arr = arr.toarray()
    else:
        arr = np.array(arr, dtype=float)
    return arr

X_train = to_float_matrix(X_train_transformed)
X_test = to_float_matrix(X_test_transformed)
y_train = np.ravel(y_train)
y_test = np.ravel(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [0]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(max_iter=300)
log_reg.fit(X_train, y_train)

log_reg_score = log_reg.score(X_test, y_test)
log_reg_score

In [0]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train, y_train)

rf_score = rf.score(X_test, y_test)
rf_score

In [0]:
results = {
    "Logistic Regression baseline": log_reg_score,
    "Random Forest baseline": rf_score
}
results

##Baseline Model Analysis

In this baseline evaluation, the Logistic Regression model performed slightly better with an accuracy of 95.11%, compared to 95.09% for the Random Forest. While Logistic Regression was marginally more accurate here, Random Forest often proves more stable for noisy sensor data because its ensemble nature (using multiple decision trees) is less likely to be "tricked" by individual outliers or sensor glitches.

The fact that the numbers are so close, and so high, leads me to wonder if the distance_cm feature provides a very clear linear signal for a "step," or if the dataset is well-balanced. It is important to test these models before deployment because an untested model could provide false health metrics; a wrong prediction could affect patients relying on accurate step counts for rehabilitation or elderly monitoring. Therefore, fairness matters in data science just as it does in discipleship because we have a responsibility to ensure our tools serve everyone equitably. Just as we are called to treat all people with integrity, our models must not harbor "hidden" biases that disadvantage certain users based on their device data.

In [0]:
import os
import joblib
from datetime import datetime

# Create a unique folder name (prevents overwriting files)
run_id = datetime.now().strftime("%Y%m%d_%H%M%S")
base_dir = f"/Workspace/Users/stef4@ensign.edu/stedi_models/{run_id}"
os.makedirs(base_dir, exist_ok=True)

# Save trained models
joblib.dump(log_reg, f"{base_dir}/log_reg.joblib")
joblib.dump(rf, f"{base_dir}/random_forest.joblib")

# Save accuracy information (metadata)
metadata = {
    "run_id": run_id,
    "logistic_regression_accuracy": float(log_reg_score),
    "random_forest_accuracy": float(rf_score),
}

joblib.dump(metadata, f"{base_dir}/metadata.joblib")

base_dir


In [0]:
import shutil
zip_path = f"/Workspace/Users/stef4@ensign.edu/stedi_models/{run_id}.zip"
shutil.make_archive(zip_path.replace(".zip", ""), "zip", base_dir)
zip_path

##5.3 Trained ML Models: Hyperparameter Tuning

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

log_reg_params = {
    "C": [0.01, 0.1, 1, 10],
    "penalty": ["l2"],
    "solver": ["lbfgs", "liblinear"]
}

log_reg_grid = GridSearchCV(
    LogisticRegression(max_iter=300),
    log_reg_params,
    cv=3,
    scoring="accuracy"
)

log_reg_grid.fit(X_train, y_train)

log_reg_best_params = log_reg_grid.best_params_
log_reg_best_score = log_reg_grid.best_score_

log_reg_best_params, log_reg_best_score

In [0]:
from sklearn.ensemble import RandomForestClassifier

rf_params = {
    "n_estimators": [50, 100, 200],
    "max_depth": [None, 5, 10, 20],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2]
}

rf_grid = GridSearchCV(
    RandomForestClassifier(),
    rf_params,
    cv=3,
    scoring="accuracy",
    n_jobs=-1
)

rf_grid.fit(X_train, y_train)

rf_best_params = rf_grid.best_params_
rf_best_score = rf_grid.best_score_

rf_best_params, rf_best_score

In [0]:
results = {
    "Logistic Regression (tuned)": log_reg_best_score,
    "Random Forest (tuned)": rf_best_score
}
results

In [0]:
# Choose the better model based on best_score_
if rf_best_score > log_reg_best_score:
    best_model = rf_grid.best_estimator_
    best_model_name = "Random Forest"
else:
    best_model = log_reg_grid.best_estimator_
    best_model_name = "Logistic Regression"

best_model_name, best_model

In [0]:
save_path = "/Workspace/Users/stef4@ensign.edu/stedi_models/stedi_best_model.pkl"
joblib.dump(best_model, save_path)

In [0]:
print(f"Successfully saved the {best_model_name} model to {save_path}")

##5.3 Model Evaluation Report

After running hyperparameter tuning, both the Logistic Regression and Random Forest models achieved an identical cross-validation accuracy of 95.11%. This suggests that the signal in the STEDI sensor data is strong enough that even a linear model can capture it as effectively as a complex ensemble of trees. I chose to proceed with the [Random Forest / Logistic Regression] as my final model.

##Ethics Reflection

Hyperparameter tuning can accidentally introduce bias if we optimize solely for a single global metric like accuracy. For example, a specific setting might increase the overall score by better predicting the majority group while significantly decreasing performance for a smaller demographic. This creates a "hidden" unfairness that documentation and transparency help reveal. Transparency is essential because it allows others to audit our choices and ensure the model serves everyone equitably. The gospel principle of Honest Evaluation reminds us that "by small and simple things are great things brought to pass" (Alma 37:6); being truthful about our model’s limitations is just as important as reporting its successes. We have a responsibility to seek light and truth in our data, ensuring our technical work reflects integrity and accountability.


In [0]:
# Ensure model is defined as your tuned winner
model = best_model 

try:
    feature_names = pipeline.named_steps["preprocess"].get_feature_names_out()
    print("Ready! Feature names loaded.")
except:
    print("Note: Using generic feature names.")

In [0]:
import numpy as np

importances = model.feature_importances_
importance_order = np.argsort(importances)[::-1]

# Get feature names if available
try:
    feature_names = pipeline.named_steps["preprocess"].get_feature_names_out()
except:
    feature_names = [f"feature_{i}" for i in range(X_train.shape[1])]

for idx in importance_order[:10]:
    print(feature_names[idx], ":", importances[idx])

###Analysis: Feature Importance

Do the most important features make sense?

Yes, it makes sense that distance_cm is a primary indicator. If the STEDI device is measuring the distance from a sensor to a person's leg or the floor, that distance will change drastically and consistently every time a step is taken. The fact that different sensor types (gyroscope vs. accelerometer) also appear shows the model is using motion data, but they are dwarfed by the distance metric.

Are there any surprises?

The biggest surprise is how much the model relies on distance_cm. Having one feature account for over 92% of the importance is unusual. It suggests the model has found a "shortcut." Also, it’s interesting that specific device_id values (like spotter-14) show up in the top 10. Ideally, a model should predict steps based on how a person moves, not which specific device they are using.

Would you trust predictions made with this importance pattern?

I would trust them, but with caution. Because the model is so dependent on one feature, if that distance sensor gets dusty, blocked, or glitches, the entire model's accuracy will likely collapse. For a high-stakes informatics application (like at UCLA or UCI), we usually prefer a more balanced model that uses multiple sensors (accelerometer + gyro + distance) so that there is "redundancy" if one sensor fails.

In [0]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.barh([feature_names[i] for i in importance_order[:10]],
         importances[importance_order[:10]])
plt.xlabel("Importance")
plt.title("Top Global Feature Importance")
plt.gca().invert_yaxis()
plt.show()

In [0]:
%pip install shap

In [0]:
import shap
shap.initjs()

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

In [0]:
# Verify the output
print(f"SHAP values calculated. Type: {type(shap_values)}")

In [0]:
shap.summary_plot(shap_values[...,1], X_test, feature_names=feature_names, rng=42)

###SHAP Summary Plot Observations
Match with Global Importance: The SHAP plot confirms the global importance chart; num__distance_cm is overwhelmingly the most influential feature, as it shows the widest spread of SHAP values.

Direction of Influence: * For num__distance_cm, blue dots (low distance values) are clustered on the right side of the center line, meaning lower distances push the model toward predicting a step.

Red dots (high distance values) are mostly on the left, pushing the prediction toward no_step.

Unexpected Influences: It is surprising that the specific device_id (like spotter-14 and spotter-26) has a visible impact. In a robust model, the physical movement (accelerometer/gyro) should matter more than which specific hardware is being used, suggesting the model might be slightly overfitted to specific devices.

In [0]:
i = 0  # choose any index you like

shap.force_plot(explainer.expected_value[1],
                shap_values[...,1][i],
                X_test[i],
                feature_names=feature_names,
                matplotlib=True)

###SHAP Force Plot Interpretation
Do you understand the explanation?
Yes. The force plot shows the "tug-of-war" between features that push the prediction away from the base value. The red arrows (like cat__sensor_type_accelerometer) are pushing the probability "higher" toward a step, while the blue arrows (like num__distance_cm and a specific device_id) are pulling it "lower." The final result is the bold value of 0.94.

Do features that push toward “step” match your expectations?
Mostly, yes. Seeing the accelerometer as a positive (red) force makes sense, as physical movement is the primary indicator of a step. However, it is interesting that for this specific instance, the distance_cm value is actually acting as a negative (blue) force, meaning this particular distance reading made the model less certain it was a step compared to the average.

Would you reach the same conclusion if you looked at the data yourself?
If I saw a high reading from the accelerometer (value = 1.0) alongside a distance reading of ~0.58, I would likely agree that some form of movement is occurring. However, because the distance feature is so dominant globally, it’s harder for a human to weigh these small decimal differences as precisely as the model does. The visualization helps bridge that gap by showing exactly how the model balances the sensor types against the distance.

###Final Reflection Questions
Since you are at the end, here is a concise block to answer the final reflection section:

Global Insight: The num__distance_cm feature is the most important overall, likely because the proximity of the user's leg to the sensor provides a very clean signal for walking patterns.

Local Insight: The SHAP force plot revealed that while the accelerometer pushed the prediction up, the distance and device ID pulled it slightly down for this specific row, resulting in a 94% probability.

Human Intuition Check: The logic mostly matches; movement (accelerometer) should predict a step. However, the heavy reliance on a single distance feature might be a "shortcut" that a human might be more skeptical of in varied environments.

Dashboard Preparation: I plan to include the Global Feature Importance bar chart and the SHAP Summary Plot. These provide a clear "personality portrait" of the model for the Week 7 dashboard.